In [5]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk import word_tokenize, FreqDist
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from nltk.classify import ClassifierI
from statistics import mode
nltk.download('movie_reviews')


[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/abdullahahmed/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [6]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = [classifier.classify(features)
                 for classifier in self._classifiers]
        return mode(votes)

    def confidence(self, features):
        votes = [classifier.classify(features)
                 for classifier in self._classifiers]
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf


In [7]:
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)

all_words = []
for w in movie_reviews.words():
    all_words.append(w.lower())

all_words = FreqDist(all_words)

word_features = list(all_words.keys())[:3000]


def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features


featuresets = [(find_features(rev), category) for (rev, category) in documents]

train_set, test_set = train_test_split(featuresets, test_size=0.3)

classifier = SklearnClassifier(SVC(kernel='linear'))
classifier.train(train_set)

print("Classifier accuracy percent:",
      (nltk.classify.accuracy(classifier, test_set)) * 100)


Classifier accuracy percent: 80.33333333333333
